# Sound propagation model data processing

This notebook brings in the sound propagation modeling results and links them to the occurrence records as produced from `presence_to_occurrence.ipynb`.

Sound propagation modeling data are available on Google Cloud at https://console.cloud.google.com/storage/browser/noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models;tab=objects?prefix=&forceOnObjectsSortingFiltering=false

We can use the [`gutil cp`](https://cloud.google.com/storage/docs/gsutil/commands/cp) command to recursively download files from Google Cloud to a local directory.

```
gutil cp -r gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/
```

But we only want the netCDF data and we only want the `sound_propagation` value.

```
ds = xr.open_dataset('SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc')
ds.variables['listening_range']

Out[6]:
<xarray.Variable (month: 1)>
array([2815])
Attributes:
    long_name:    distance_from_hydrophone_to_zero_SNR
    Description:  The median distance from the hydrophone to a zero signal-to...
    units:        m
```



In [3]:
import pandas as pd
import xarray as xr

# Function to download public files.

From https://cloud.google.com/storage/docs/access-public-data#storage-download-public-object-python

```
conda install google-cloud-storage
```

https://console.cloud.google.com/storage/browser/_details/noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc;tab=live_object

In [2]:
# Download the readme for the noaa-passive-bioacoustic bucket

from google.cloud import storage
import os

storage_client = storage.Client.create_anonymous_client()

bucket_name = 'noaa-passive-bioacoustic'
delimiter='/'
bucket=storage_client.get_bucket(bucket_name)
blobs=bucket.list_blobs(delimiter=delimiter) #List all objects that satisfy the filter.


for blob in blobs:
    print(blob.name)
    if not os.path.exists(blob.name):
        blob.download_to_filename(blob.name)

README.pdf


In [3]:
from IPython.display import IFrame

IFrame(blob.name, width=900, height=1200)

Install `gsutil` from https://cloud.google.com/storage/docs/gsutil_install

and recusively download netCDF files from `noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models`

gsutil uri for one of the datasets is:
```
gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc
```

Data citation:
```
NOAA National Centers for Environmental Information. 2017. Passive Acoustic Data Collection. NOAA National Centers for Environmental Information.
https://doi.org/10.25921/PF0H-SQ72. access date
```

In [21]:
# for one station
#!gcloud storage ls gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/sb03/**/*.nc
    
# for all stations

temp = !gcloud storage ls gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/**/*.nc

print(f'Found {len(temp)} files:')    

for file in temp:
    print(file.split('/')[-1])

Found 924 files:
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Jan_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Jul_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Oct_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Apr_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Jan_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Jul_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Oct_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Apr_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Jan_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Jul_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Oct_radarformat_highres.nc

In [1]:
## BE CAREFUL WITH THIS >900 FILES!
#  Operation completed over 924 objects/103.2 GiB.       
# ==> NOTE: You are performing a sequence of gsutil operations that may
# run significantly faster if you instead use gsutil -m cp ... Please
# see the -m section under "gsutil help options" for further information
# about when gsutil -m can be advantageous.
#
# !gsutil cp gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/**/*.nc data\sound_propagation\

Copying gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc...
/ [0 files][    0.0 B/152.8 MiB]                                                
-
- [0 files][ 14.7 MiB/152.8 MiB]                                                
\
|
| [0 files][ 71.9 MiB/152.8 MiB]                                                
/
/ [0 files][130.9 MiB/152.8 MiB]                                                
-
- [1 files][152.8 MiB/152.8 MiB]                                                
Copying gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Jan_radarformat_highres.nc...
- [1 files][152.8 MiB/207.3 MiB]                                                
\
\ [1 files][164.6 MiB/207.3 MiB]                                                
|
/
/ [

In [6]:
# Might be able to use xarray to grab data from gc:

# url = 'gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc'

# xr.open_dataset(url,
#                 engine="netcdf4"
#                )

# Match occurrence data with Sound propagation model

Now we can bring in the occurrence data and determine how to link the two resources together.

Reminder, occurrence data was collected from https://coastwatch.pfeg.noaa.gov/erddap/search/index.html?page=1&itemsPerPage=1000&searchFor=noaaSanctSound

In [4]:
df_occur = pd.read_csv('data/occurrence.zip', compression='zip')

df_occur.head(5)

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,occurrenceID
0,2018-11-01,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,CI01_01_bocaccio_2018-11-01
1,2018-11-01,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,CI01_01_humpback_whale_2018-11-01
2,2018-11-01,CI01_01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300,CI01_01_plainfin_midshipman_2018-11-01
3,2018-11-01,CI02_01,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,34.0856,-120.5232,63,CI02_01_blue_whale_2018-11-01
4,2018-11-01,CI02_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0856,-120.5232,300,CI02_01_bocaccio_2018-11-01


Let's collect unique station and locality identifiers to match to propagation results.

In [5]:
df_occur['occurrenceID'].str.split('_',expand=True)[0].unique()

array(['CI01', 'CI02', 'CI04', 'CI05', 'SB01', 'SB02', 'SB03', 'MB03',
       'MB01', 'MB02', 'GR01', 'FK01', 'FK03', 'FK02', 'OC02', 'OC01',
       'GR03', 'GR02', 'OC04', 'PM05', 'CI03', 'OC03', 'HI03', 'HI04'],
      dtype=object)

# Investigate downloaded sound propagation files

We only downloaded a subset of the propagation model data for testing. There are **924** files, so we should explore how we might be able to do this work without downloading all the data.

What does our sound propagation model output look like? Let's look at the first file that we downloaded.

In [6]:
import os

directory = 'data/sound_propagation/'

fname = os.listdir(directory)[0]

print(fname,'\n')

ds = xr.open_dataset(directory+fname, engine='netcdf4')

ds.info()

SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc 

xarray.Dataset {
dimensions:
	month = 1 ;
	depth = 1 ;
	bearing = 361 ;
	range = 13853 ;

variables:
	float64 month(month) ;
		month:long_name = month_of_climatological_sound_speed_profiles ;
		month:description = month # for GDEM sound speed profiles: 1-Jan, 2-Feb etc ;
		month:units = 1 ;
	float64 depth(depth) ;
		depth:long_name = sound_source_depth ;
		depth:description = Depth of the sound source ;
		depth:units = m ;
	float64 bearing(bearing) ;
		bearing:axis = Y ;
		bearing:long_name = true_north_bearing_from_hydrophone ;
		bearing:units = degrees_true ;
	float64 range(range) ;
		range:axis = X ;
		range:long_name = range_away_from_hydrophone ;
		range:units = km ;
	|S4 site() ;
		site:long_name = SanctSound_site_name ;
	float64 latitude(bearing, range) ;
		latitude:standard_name = latitude ;
		latitude:units = degrees_north ;
	float64 longitude(bearing, range) ;
		longitude:standard_name = longi

We know we want the data from the variable `listening_range`.

In [7]:
ds['listening_range']

<xarray.DataArray 'listening_range' (month: 1)>
[1 values with dtype=int32]
Coordinates:
  * month    (month) float64 4.0
Attributes:
    long_name:    distance_from_hydrophone_to_zero_SNR
    Description:  The median distance from the hydrophone to a zero signal-to...
    units:        m

Grab all the `listening_range` data (in meters) with filenames to see if we can match to the occurrence data

In [8]:

df_listening_range = pd.DataFrame()

fnames = os.listdir(directory)

for fname in fnames:
    
    with xr.open_dataset(directory+fname, engine='netcdf4') as ds:
    
        df_temp = ds[['listening_range','depth','site']].to_dataframe().reset_index()

        # add additional data
        df_temp['fname'] = fname
        df_temp['site'] = df_temp['site'].values[0].decode('utf-8')
        df_temp['freq_Hz'] = int(ds.SoundSourcefrequency.replace('Hz','').strip())
        df_temp['hydrophone_depth_m'] = ds.HydrophoneDepth.replace('m','').strip()
        df_temp.rename(columns={'listening_range':'listening_range_m','month':'climatology'}, inplace=True)
        
        df_listening_range = pd.concat([df_listening_range, df_temp])
        
df_listening_range.sample(5)

,climatology,depth,listening_range_m,site,fname,freq_Hz,hydrophone_depth_m
0,1.0,20.0,7056,GR02,SanctSound_GR02_propmodeling_SD0020m_SL170dB_F...,300,17.0
0,7.0,20.0,13918,HI05,SanctSound_HI05_propmodeling_SD0020m_SL170dB_F...,300,48.0
0,4.0,20.0,51208,HI02,SanctSound_HI02_propmodeling_SD0020m_SL192dB_F...,63,63.0
0,1.0,20.0,29456,GR03,SanctSound_GR03_propmodeling_SD0020m_SL192dB_F...,63,16.0
0,1.0,1.0,37,CI04,SanctSound_CI04_propmodeling_SD0001m_SL165dB_F...,1000,152.5


Print out the dataframe to share via chat

In [12]:
# pd.set_option('display.max_colwidth', None)

# columns = [ 'site','month','freq_Hz', 'depth', 'listening_range_m']

# print(df_listening_range.sort_values(by=columns, ascending=True).to_csv(columns=columns,index=False))

Find occurrence records associated with the station `CI01`.

In [9]:
df_occur.loc[df_occur['occurrenceID'].str.contains('CI01')]

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,occurrenceID
0,2018-11-01,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,CI01_01_bocaccio_2018-11-01
1,2018-11-01,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,CI01_01_humpback_whale_2018-11-01
2,2018-11-01,CI01_01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300,CI01_01_plainfin_midshipman_2018-11-01
9,2018-11-02,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,CI01_01_bocaccio_2018-11-02
10,2018-11-02,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,CI01_01_humpback_whale_2018-11-02
...,...,...,...,...,...,...,...,...,...,...,...
21747,2021-09-07,CI01_08,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0436,-120.0803,300,CI01_08_plainfin_midshipman_2021-09-07
21762,2021-09-08,CI01_08,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0436,-120.0803,300,CI01_08_plainfin_midshipman_2021-09-08
21777,2021-09-09,CI01_08,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0436,-120.0803,300,CI01_08_plainfin_midshipman_2021-09-09
21792,2021-09-10,CI01_08,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0436,-120.0803,300,CI01_08_plainfin_midshipman_2021-09-10


Now find the **listening ranges** for station `CI01` for the January climatology (month = 1.0).

In [10]:
df_listening_range.loc[(df_listening_range['site']=='CI01') & (df_listening_range['climatology']==1.0)].sort_values(by=['freq_Hz'])

,climatology,depth,listening_range_m,site,fname,freq_Hz,hydrophone_depth_m
0,1.0,15.0,4861,CI01,SanctSound_CI01_propmodeling_SD0015m_SL189dB_F...,20,17.5
0,1.0,20.0,15001,CI01,SanctSound_CI01_propmodeling_SD0020m_SL192dB_F...,63,17.5
0,1.0,10.0,18270,CI01,SanctSound_CI01_propmodeling_SD0010m_SL185dB_F...,125,17.5
0,1.0,20.0,9492,CI01,SanctSound_CI01_propmodeling_SD0020m_SL170dB_F...,300,17.5
0,1.0,1.0,4593,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,1000,17.5
0,1.0,1.0,3028,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,5000,17.5
0,1.0,19.0,7325,CI01,SanctSound_CI01_propmodeling_SD0019m_SL176dB_F...,12000,17.5


Let's look at the SanctSound website and see how we might be able to link these together.

https://sanctsound.portal.axds.co/#sanctsound/sanctuary/channel-islands/site/CI01

Since propagation model data are separated into quarterly observations on months 1, 4, 7, and 10. We can use pandas to group by quarters starting in January.

In [11]:
df_occur['eventDate'] = pd.to_datetime(df_occur['eventDate'])

df_occur['site'] = df_occur['occurrenceID'].str.split("_",expand=True)[0]

df_occur.sample(5)

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,propagationFrequency,occurrenceID,site
3235,2019-04-18,CI02_02,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.08540,-120.523300,300,CI02_02_humpback_whale_2019-04-18,CI02
13945,2020-05-25,MB03_03,dolphin,urn:lsid:marinespecies.org:taxname:2688,Cetacea,Infraorder,Animalia,36.37021,-122.314903,5000,MB03_03_dolphin_2020-05-25,MB03
18339,2020-12-03,CI02_06,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.08523,-120.095700,300,CI02_06_humpback_whale_2020-12-03,CI02
18610,2020-12-15,CI02_06,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.08523,-120.095700,300,CI02_06_humpback_whale_2020-12-15,CI02
22089,2021-09-28,SB02_18,dolphin,urn:lsid:marinespecies.org:taxname:2688,Cetacea,Infraorder,Animalia,42.47146,-70.241800,5000,SB02_18_dolphin_2021-09-28,SB02


Map the months to the appropriate climatology

Months | Climatology
-------|------------
1,2,3 | 1
4,5,6 | 4
7,8,9 | 7
10,11,12 | 10

See this gist for confirmation they line up: <https://gist.github.com/ocefpaf/412a6ddcfa3524862160653f1718da5f>

In [12]:
df_occur['climatology'] = pd.Series(dtype=float)

mask = (df_occur['eventDate'].dt.quarter==1)

df_occur.loc[mask,'climatology'] = 1

mask = (df_occur['eventDate'].dt.quarter==2)

df_occur.loc[mask,'climatology'] = 4

mask = (df_occur['eventDate'].dt.quarter==3)

df_occur.loc[mask,'climatology'] = 7

mask = (df_occur['eventDate'].dt.quarter==4)

df_occur.loc[mask, 'climatology'] = 10

 Use [DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) to combine the datasets together.
 
 First, however, we pick an appropriate frequency. 
 
 From <https://dosits.org/galleries/audio-gallery/marine-mammals/baleen-whales/blue-whale/>
 
> Blue whales produce stereotyped calls throughout the year. These calls have been described as pulses, grunts, groans, and moans, and are typically in the 15-40 Hz range

In this example we choose 20 Hz.

 **Note: We have not copied all of the listening range data over, so we will have some gaps.**

In [13]:
df_occur['vernacularName'].unique()

array(['bocaccio', 'humpback whale', 'plainfin midshipman', 'blue whale',
       'fin whale', 'dolphin', 'atlantic cod', 'minke whale',
       'north atlantic right whale', 'sei whale', 'pinniped',
       'black grouper', 'red grouper', 'killer whale'], dtype=object)

In [14]:
df_occur.rename(columns={'propagationFrequency':'freq_Hz'}, inplace=True)

df_occur

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,freq_Hz,occurrenceID,site,climatology
0,2018-11-01,CI01_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0438,-120.0811,300,CI01_01_bocaccio_2018-11-01,CI01,10.0
1,2018-11-01,CI01_01,humpback whale,urn:lsid:marinespecies.org:taxname:137092,Megaptera novaeangliae,Species,Animalia,34.0438,-120.0811,300,CI01_01_humpback_whale_2018-11-01,CI01,10.0
2,2018-11-01,CI01_01,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.0438,-120.0811,300,CI01_01_plainfin_midshipman_2018-11-01,CI01,10.0
3,2018-11-01,CI02_01,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,34.0856,-120.5232,63,CI02_01_blue_whale_2018-11-01,CI02,10.0
4,2018-11-01,CI02_01,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,34.0856,-120.5232,300,CI02_01_bocaccio_2018-11-01,CI02,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22923,2021-12-31,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03_07_fin_whale_2021-12-31,CI03,10.0
22924,2022-01-02,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03_07_fin_whale_2022-01-02,CI03,1.0
22925,2022-01-03,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03_07_fin_whale_2022-01-03,CI03,1.0
22926,2022-01-04,CI03_07,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,33.4868,-119.0159,20,CI03_07_fin_whale_2022-01-04,CI03,1.0


# Do the combining

In [15]:
df_combined = df_occur.merge(df_listening_range,how='left', on=['site','climatology','freq_Hz'], indicator=True)

df_combined.sample(10)

,eventDate,station,vernacularName,scientificNameID,scientificName,taxonRank,kingdom,decimalLatitude,decimalLongitude,freq_Hz,occurrenceID,site,climatology,depth,listening_range_m,fname,hydrophone_depth_m,_merge
22019,2021-09-24,SB01_17,dolphin,urn:lsid:marinespecies.org:taxname:2688,Cetacea,Infraorder,Animalia,42.438300,-70.545683,5000,SB01_17_dolphin_2021-09-24,SB01,7.0,1.0,4047,SanctSound_SB01_propmodeling_SD0001m_SL165dB_F...,46.0,both
6595,2019-09-10,MB02_03,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,36.649500,-121.908400,300,MB02_03_bocaccio_2019-09-10,MB02,7.0,20.0,8741,SanctSound_MB02_propmodeling_SD0020m_SL170dB_F...,64.5,both
11967,2020-03-19,HI04_02,dolphin,urn:lsid:marinespecies.org:taxname:2688,Cetacea,Infraorder,Animalia,22.263472,-159.586389,5000,HI04_02_dolphin_2020-03-19,HI04,1.0,1.0,4241,SanctSound_HI04_propmodeling_SD0001m_SL165dB_F...,81.0,both
15695,2020-08-29,CI01_05,plainfin midshipman,urn:lsid:marinespecies.org:taxname:275658,Porichthys notatus,Species,Animalia,34.046300,-120.080730,300,CI01_05_plainfin_midshipman_2020-08-29,CI01,7.0,20.0,8862,SanctSound_CI01_propmodeling_SD0020m_SL170dB_F...,17.5,both
22830,2021-11-19,SB02_18,dolphin,urn:lsid:marinespecies.org:taxname:2688,Cetacea,Infraorder,Animalia,42.471460,-70.241800,5000,SB02_18_dolphin_2021-11-19,SB02,10.0,1.0,3297,SanctSound_SB02_propmodeling_SD0001m_SL165dB_F...,66.0,both
6263,2019-08-31,CI04_03,blue whale,urn:lsid:marinespecies.org:taxname:137090,Balaenoptera musculus,Species,Animalia,33.848880,-120.117000,63,CI04_03_blue_whale_2019-08-31,CI04,7.0,20.0,44328,SanctSound_CI04_propmodeling_SD0020m_SL192dB_F...,152.5,both
8591,2019-11-08,CI04_03,bocaccio,urn:lsid:marinespecies.org:taxname:274833,Sebastes paucispinis,Species,Animalia,33.848880,-120.117000,300,CI04_03_bocaccio_2019-11-08,CI04,10.0,20.0,2361,SanctSound_CI04_propmodeling_SD0020m_SL170dB_F...,152.5,both
12450,2020-04-03,SB01_08,sei whale,urn:lsid:marinespecies.org:taxname:137088,Balaenoptera borealis,Species,Animalia,42.436927,-70.545857,63,SB01_08_sei_whale_2020-04-03,SB01,4.0,20.0,49282,SanctSound_SB01_propmodeling_SD0020m_SL192dB_F...,46.0,both
20269,2021-04-12,SB01_15,dolphin,urn:lsid:marinespecies.org:taxname:2688,Cetacea,Infraorder,Animalia,42.438427,-70.545420,5000,SB01_15_dolphin_2021-04-12,SB01,4.0,1.0,2630,SanctSound_SB01_propmodeling_SD0001m_SL165dB_F...,46.0,both
7827,2019-10-17,SB02_06,fin whale,urn:lsid:marinespecies.org:taxname:137091,Balaenoptera physalus,Species,Animalia,42.470960,-70.242305,20,SB02_06_fin_whale_2019-10-17,SB02,10.0,15.0,48282,SanctSound_SB02_propmodeling_SD0015m_SL189dB_F...,66.0,both


In [16]:
df_combined.rename(columns={'listening_range_m':'coordinateUncertaintyMeters'}, inplace=True)

In [17]:
df_combined.to_csv('data/occurrence_w_coordinateUncertainty.zip', compression='zip')